In [1]:
# to fix trame issue
import nest_asyncio
nest_asyncio.apply()

In [2]:
import sympy
import underworld3 as uw
import numpy as np
import os 
import mpi4py

os.environ["UW_TIMING_ENABLE"] = "1"
os.environ["UW_JITNAME"] = "TESTF"

# These are tested by test_001_meshes.py

mesh = uw.meshing.UnstructuredSimplexBox(
    cellSize=0.1, regular=True, qdegree=2, refinement=0
)

if mesh.dim == 2:
    x, y = mesh.X
else:
    x, y, z = mesh.X

u = uw.discretisation.MeshVariable("U", mesh, mesh.dim, 
                                   vtype=uw.VarType.VECTOR, 
                                   degree=2, varsymbol=r"\mathbf{u}",
)
p = uw.discretisation.MeshVariable("P",
    mesh, 1, vtype=uw.VarType.SCALAR, degree=1, continuous=True, 
                                   varsymbol=r"\mathbf{p}", 
)

stokes = uw.systems.Stokes(mesh, velocityField=u, pressureField=p, verbose=True)
stokes.constitutive_model = (
    uw.constitutive_models.ViscoElasticPlasticFlowModel
)
stokes.constitutive_model.Parameters.shear_viscosity_0 = 1

stokes.petsc_options["snes_type"] = "newtonls"
stokes.petsc_options["ksp_type"] = "fgmres"
stokes.petsc_options["ksp_monitor"] = None
stokes.petsc_options["snes_monitor"] = None
stokes.tolerance = 1.0e-3

# stokes.petsc_options.setValue("fieldsplit_velocity_pc_type", "mg")
stokes.petsc_options.setValue("fieldsplit_velocity_pc_mg_type", "kaskade")
stokes.petsc_options.setValue("fieldsplit_velocity_pc_mg_cycle_type", "w")

stokes.petsc_options["fieldsplit_velocity_mg_coarse_pc_type"] = "svd"
stokes.petsc_options[f"fieldsplit_velocity_ksp_type"] = "fcg"
stokes.petsc_options[f"fieldsplit_velocity_mg_levels_ksp_type"] = "chebyshev"
stokes.petsc_options[f"fieldsplit_velocity_mg_levels_ksp_max_it"] = 7
stokes.petsc_options[f"fieldsplit_velocity_mg_levels_ksp_converged_maxits"] = None

stokes.petsc_options.setValue("fieldsplit_pressure_pc_type", "gamg")
stokes.petsc_options.setValue("fieldsplit_pressure_pc_mg_type", "multiplicative")
stokes.petsc_options.setValue("fieldsplit_pressure_pc_mg_cycle_type", "v")

if mesh.dim == 2:
    stokes.bodyforce = sympy.Matrix([0.0, 1.0*sympy.sin(x)])

    stokes.add_natural_bc((0.0, -u[1].sym), "Top")
    # stokes.add_dirichlet_bc((sympy.oo, 0.0), "Top")
    stokes.add_dirichlet_bc((sympy.oo,0.0), "Bottom")
    stokes.add_dirichlet_bc((0.0,sympy.oo), "Left")
    stokes.add_dirichlet_bc((0.0,sympy.oo), "Right")

else:
    stokes.bodyforce = sympy.Matrix([0, sympy.sin(x), 0])

    stokes.add_essential_bc([0.0, 0.0, 0.0], "Top")
    stokes.add_essential_bc([0.0, 0.0, 0.0], "Bottom")
    stokes.add_essential_bc([0.0, 0.0, 0.0], "Left")
    stokes.add_essential_bc([0.0, 0.0, 0.0], "Right")    
    stokes.add_essential_bc([0.0, 0.0, 0.0], "Front")
    stokes.add_essential_bc([0.0, 0.0, 0.0], "Back")


In [3]:
with uw.utilities.CaptureStdout(split=False) as captured_dm_view:
    mesh.dm.view()


In [4]:
stokes.solve(verbose=False, debug=False)
stokes.dm.ds.view()

## assert stokes.snes.getConvergedReason() > 0

Stokes: Jacobians complete, now compile
Setting bc 0 (natural)
 - component: [0 1]
 - boundary:   Top
 - fn:         Matrix([[0], [-\mathbf{u}_{ 1 }(N.x, N.y)]]) 
Setting bc 0 (essential)
 - component: [1]
 - boundary:   Bottom
 - fn:         Matrix([[oo], [0]]) 
Setting bc 1 (essential)
 - component: [0]
 - boundary:   Left
 - fn:         Matrix([[0], [oo]]) 
Setting bc 2 (essential)
 - component: [0]
 - boundary:   Right
 - fn:         Matrix([[0], [oo]]) 
  0 SNES Function norm 2.942451402048e-02 
    Residual norms for  solve.
    0 KSP Residual norm 2.942451402048e-02 
    1 KSP Residual norm 1.238283182747e-06 
    2 KSP Residual norm 3.281927123375e-10 
  1 SNES Function norm 4.473251006544e-03 
    Residual norms for  solve.
    0 KSP Residual norm 4.473251006544e-03 
    1 KSP Residual norm 3.806036941718e-07 
    2 KSP Residual norm 7.041457696798e-12 
  2 SNES Function norm 6.929441772867e-04 
    Residual norms for  solve.
    0 KSP Residual norm 6.929441772867e-04 
    1 K

In [5]:
for pen in [10, 33, 100, 330, 1000]:
    stokes.natural_bcs = []
    stokes.dm = None
    stokes.add_natural_bc((0.0, pen*u[1].sym), "Top")

    
    stokes.solve(zero_init_guess=False, _force_setup=True)

    

Stokes: Jacobians complete, now compile
Setting bc 0 (natural)
 - component: [0 1]
 - boundary:   Top
 - fn:         Matrix([[0], [10*\mathbf{u}_{ 1 }(N.x, N.y)]]) 
Setting bc 0 (essential)
 - component: [1]
 - boundary:   Bottom
 - fn:         Matrix([[oo], [0]]) 
Setting bc 1 (essential)
 - component: [0]
 - boundary:   Left
 - fn:         Matrix([[0], [oo]]) 
Setting bc 2 (essential)
 - component: [0]
 - boundary:   Right
 - fn:         Matrix([[0], [oo]]) 
  0 SNES Function norm 5.817436137642e-02 
    Residual norms for  solve.
    0 KSP Residual norm 5.817436137642e-02 
    1 KSP Residual norm 4.644705720326e-06 
    2 KSP Residual norm 7.741462195277e-11 
  1 SNES Function norm 1.456187362015e-02 
    Residual norms for  solve.
    0 KSP Residual norm 1.456187362015e-02 
    1 KSP Residual norm 1.319167806290e-06 
    2 KSP Residual norm 4.816825473015e-11 
  2 SNES Function norm 3.646466410109e-03 
    Residual norms for  solve.
    0 KSP Residual norm 3.646466410109e-03 
    1

In [6]:
if uw.mpi.size == 1:

    import pyvista as pv
    import underworld3.visualisation as vis

    pvmesh = vis.mesh_to_pv_mesh(mesh)
    pvmesh.point_data["V"] = vis.vector_fn_to_pv_points(pvmesh, u.sym)
    pvmesh.point_data["P"] = vis.scalar_fn_to_pv_points(pvmesh, p.sym)
    velocity_points = vis.meshVariable_to_pv_cloud(u)
    velocity_points.point_data["V"] = vis.vector_fn_to_pv_points(velocity_points, u.sym)

    pl = pv.Plotter(window_size=[1000, 1000])
    pl.add_axes()

    pl.add_mesh(
        pvmesh,
        cmap="coolwarm",
        edge_color="Black",
        show_edges=True,
        scalars="P",
        use_transparency=False,
        opacity=1.0,
    )

    # pl.add_mesh(pvmesh, cmap="coolwarm", edge_color="Black", show_edges=True, scalars="T",
    #               use_transparency=False, opacity=1.0)

    # pl.add_arrows(pvmesh.points, pvmesh.point_data["V"], mag=10)
    pl.add_arrows(velocity_points.points, velocity_points.point_data["V"], mag=10)

    pl.show(cpos="xy")

Widget(value="<iframe src='http://localhost:52899/index.html?ui=P_0x17b7236d0_0&reconnect=auto' style='width: …

In [7]:
stokes.natural_bcs[0].fns["f0"] = "test"